# Análise de Dados de Fundos Imobiliários

## Importando Bibliotecas

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
plt.style.use('ggplot')
sns.set(context='notebook', palette='dark', color_codes=True)

from data_miner import DataFII

## Explorando e Preparando os Dados
Os dados aqui contidos foram obtidos no dia 29/07/2021 a partir dos seguintes sites:
 - Funds Explorer - www.fundsexplorer.com.br
 - Status Invest - www.statusinvest.com.br

In [2]:
# Obtem dados da pasta /data/
df = DataFII().get_FII_data()
print(df.shape)
df.head()

(357, 34)


,ticker,nome,tipo_gestao,setor,preço,dy,pct_caixa,num_cotistas,PL,PVPA,...,rent_periodo,rent_acc,VPA,dy_VPA,var_VPA,rent_VPA_periodo,rent_VPA_acc,vacancia_fisica,vacancia_financeira,qtd_ativos
0,BRLA11,FUNDO DE INVESTIMENTO IMOBILIÁRIO BRL PROP II,Passiva,Híbrido,0.0,0.0000,0.7435,84.0,1.376312e+08,NaN,...,0.00,0.00,140.98,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,EGYR11,ENERGY RESORT FDO. INVEST. IMOB,Passiva,NaN,0.0,0.0000,0.0576,21.0,9.322396e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FINF11,Infra Real State,Ativa,NaN,0.0,0.0000,0.7986,4.0,8.364985e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FCAS11,Edifício Castelo,Passiva,NaN,0.0,0.0000,186.1664,2.0,8.068422e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SHPH11,Shopping Patio Higienopolis,Passiva,Shoppings,807.0,3.4885,0.4134,3105.0,5.013048e+08,0.9803,...,0.24,-0.33,823.23,NaN,NaN,NaN,NaN,13.3,NaN,1.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 357 entries, 0 to 356
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ticker               357 non-null    object 
 1   nome                 356 non-null    object 
 2   tipo_gestao          356 non-null    object 
 3   setor                245 non-null    object 
 4   preço                356 non-null    float64
 5   dy                   356 non-null    float64
 6   pct_caixa            352 non-null    float64
 7   num_cotistas         356 non-null    float64
 8   PL                   356 non-null    float64
 9   PVPA                 302 non-null    float64
 10  liquidez_diaria_avg  284 non-null    float64
 11  cagr_dividendo       120 non-null    float64
 12  cagr_cota            113 non-null    float64
 13  liquidez_diaria      238 non-null    float64
 14  div_mes_brl          246 non-null    float64
 15  dy_1m                233 non-null    flo

O Dataframe possui 34 características/informações de **357 FIIs**.

Todos os dados já foram pré-processados pelo `data_miner.py` com o objetivo de limpar e definir a correta tipagem dos dados (os arquivos .csv na pasta `data/` estão no formato cru, ou seja, dados não processados, conforme foram obtidos na web). 

Entretanto, pode-se observar que há muitos dados faltantes. Vamos analisar caso a caso.

### Dados Faltantes

In [4]:
df.isna().sum()

ticker                   0
nome                     1
tipo_gestao              1
setor                  112
preço                    1
dy                       1
pct_caixa                5
num_cotistas             1
PL                       1
PVPA                    55
liquidez_diaria_avg     73
cagr_dividendo         237
cagr_cota              244
liquidez_diaria        119
div_mes_brl            111
dy_1m                  124
dy_3m_acc              124
dy_6m_acc              124
dy_12m_acc             124
dy_3m_avg              124
dy_6m_avg              124
dy_12m_avg             124
dy_ano                 137
var_preço              124
rent_periodo           124
rent_acc               124
VPA                    111
dy_VPA                 348
var_VPA                348
rent_VPA_periodo       348
rent_VPA_acc           348
vacancia_fisica        239
vacancia_financeira    331
qtd_ativos             111
dtype: int64

Existe uma observaçao sem nome. Buscando-a é possível observar que se trata do FII `YUFI11B` que em uma base dados aparece sem o final "B" de Balcão, linhas 354 e 356. Como o fundo possui somente 9 cotistas (filtro de número de cotistas será adicionado posteriormente), essa observação será removida.

In [5]:
i = df[df['nome'].isna()].index[0]
df.loc[i-2:i,:]

,ticker,nome,tipo_gestao,setor,preço,dy,pct_caixa,num_cotistas,PL,PVPA,...,rent_periodo,rent_acc,VPA,dy_VPA,var_VPA,rent_VPA_periodo,rent_VPA_acc,vacancia_fisica,vacancia_financeira,qtd_ativos
354,YUFI11,YUCA FDO INV. IMOB.,Ativa,NaN,0.00,0.0,1.8786,9.0,37581481.17,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
355,ZIFI11,ZION CAPITAL FUNDO DE INVESTIMENTO IMOBILIÁRIO,Ativa,NaN,1200.01,0.0,2.7883,16.0,23483861.85,1.0616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
356,YUFI11B,NaN,NaN,Residencial,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,95.11,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [6]:
df.drop(i, inplace = True)
df.isna().sum()

ticker                   0
nome                     0
tipo_gestao              0
setor                  112
preço                    0
dy                       0
pct_caixa                4
num_cotistas             0
PL                       0
PVPA                    54
liquidez_diaria_avg     72
cagr_dividendo         236
cagr_cota              243
liquidez_diaria        118
div_mes_brl            111
dy_1m                  124
dy_3m_acc              124
dy_6m_acc              124
dy_12m_acc             124
dy_3m_avg              124
dy_6m_avg              124
dy_12m_avg             124
dy_ano                 136
var_preço              124
rent_periodo           124
rent_acc               124
VPA                    111
dy_VPA                 347
var_VPA                347
rent_VPA_periodo       347
rent_VPA_acc           347
vacancia_fisica        238
vacancia_financeira    330
qtd_ativos             111
dtype: int64

Há 112 valores nulos na variável `setor`. Vamos observá-los!

In [7]:
setor_na = df.loc[df['setor'].isna()]
setor_na.head(10)

,ticker,nome,tipo_gestao,setor,preço,dy,pct_caixa,num_cotistas,PL,PVPA,...,rent_periodo,rent_acc,VPA,dy_VPA,var_VPA,rent_VPA_periodo,rent_VPA_acc,vacancia_fisica,vacancia_financeira,qtd_ativos
1,EGYR11,ENERGY RESORT FDO. INVEST. IMOB,Passiva,NaN,0.00,0.0000,0.0576,21.0,9.322396e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,FINF11,Infra Real State,Ativa,NaN,0.00,0.0000,0.7986,4.0,8.364985e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,FCAS11,Edifício Castelo,Passiva,NaN,0.00,0.0000,186.1664,2.0,8.068422e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,ANCR11B,Ancar CI,Ativa,NaN,3400.00,1.1037,0.9010,55.0,1.502356e+09,1.0209,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,BRIM11,BRIO REAL ESTATE II - FDO INV IMOB,Ativa,NaN,1200.01,0.0000,5.4897,121.0,1.204895e+08,1.2363,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,BRIP11,BRIO REAL ESTATE III,Ativa,NaN,1099.00,0.0000,19.9399,235.0,1.070182e+08,1.1126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,BTRA11,FDO INV IMOB BTG PACTUAL TERRAS AGRÍCOLAS,Ativa,NaN,96.10,0.0029,100.0140,7275.0,3.362901e+08,0.9615,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,BTSG11,BTSP I FDO. INVEST. IMOB,Ativa,NaN,0.00,0.0000,0.9152,52.0,7.261384e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,BTSI11,BTSP II FUNDO DE INVESTIMENTO IMOBILIÁRIO,Ativa,NaN,0.00,0.0000,0.3145,52.0,1.784025e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,BVAR11,Brasil Varejo,Passiva,NaN,0.00,0.0000,0.8708,69.0,6.801733e+08,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
print(list(setor_na.ticker))

['EGYR11', 'FINF11', 'FCAS11', 'ANCR11B', 'BRIM11', 'BRIP11', 'BTRA11', 'BTSG11', 'BTSI11', 'BVAR11', 'BZEL11', 'DOVL11B', 'FISD11', 'FOFT11', 'FVBI11', 'GRLV11', 'JTPR11', 'LOFT11B', 'MINT11', 'PBLV11', 'PRTS11', 'RBRM11', 'RMAI11', 'SOLR11', 'SPAF11', 'TBOF11', 'TCIN11', 'TCPF11', 'TORM13', 'TOUR11', 'VCRR11', 'VTRT11', 'VTVI11', 'VTXI11', 'XPHT12', 'ATCR11', 'ATWN11', 'VERE11', 'CJFI11', 'BMII11', 'ERPA11', 'BPRP11', 'BRHT11B', 'HBTT11', 'HCST11', 'HMOC11', 'JBFO11', 'CCRF11', 'ESTQ11', 'LATR11B', 'LKDV11', 'BLMO11', 'AQLL11', 'ARFI11B', 'FPNG11', 'FTCE11B', 'HUSI11', 'MORC11', 'CFHI11', 'BICE11', 'BTWR11', 'DLMT11', 'IBCR11', 'KEVE11', 'MTOF11', 'PEMA11', 'DAMT11B', 'DVFF11', 'NPAR11', 'RBTS11', 'RCRI11B', 'ELDO11B', 'HBCR11', 'PNDL11', 'PNPR11', 'PRZS11', 'JPPC11', 'MGLG11', 'LSPA11', 'MVFI11', 'MOFF11', 'RBIR11', 'PRSN11B', 'REIT11', 'SFND11', 'PATB11', 'QAMI11', 'RCFA11', 'RCFF11', 'SHDP11B', 'SHOP11', 'ROOF11', 'RSPD11', 'SAIC11B', 'SALI11', 'SBCL11', 'TFOF11', 'SJAU11', 'VSEC1

É possível observar que os fundos que não possuem dados para o setor, não possuem dados para várias variáveis mais a direita do DF (variáveis obtidas atraves do site Funds Explorer). Isso se deve ao fato de as fontes de dados retornarem quantidades diferentes de Fundos Imobiliários:

In [9]:
qtd_fe = len(DataFII().get_raw_data_fundsexplorer())
qtd_si = len(DataFII().get_raw_data_statusinvest())

print("Quantidade de FIIs em Funds Explorer: {}".format(qtd_fe))
print("Quantidade de FIIs em Status Invest: {}".format(qtd_si))
qtd_si - qtd_fe

Quantidade de FIIs em Funds Explorer: 246
Quantidade de FIIs em Status Invest: 356


110

Analisando esses fundos pode-se observar que muitos são fundos pequenos sem negociação em bolsa de valores ou que já fecharam.

Logo, essas observações serão excluidas:

In [10]:
df.drop(setor_na.index, inplace = True)
df.isna().sum()

ticker                   0
nome                     0
tipo_gestao              0
setor                    0
preço                    0
dy                       0
pct_caixa                2
num_cotistas             0
PL                       0
PVPA                     5
liquidez_diaria_avg      6
cagr_dividendo         138
cagr_cota              138
liquidez_diaria          7
div_mes_brl              0
dy_1m                   12
dy_3m_acc               12
dy_6m_acc               12
dy_12m_acc              12
dy_3m_avg               12
dy_6m_avg               12
dy_12m_avg              12
dy_ano                  24
var_preço               12
rent_periodo            12
rent_acc                12
VPA                      0
dy_VPA                 235
var_VPA                235
rent_VPA_periodo       235
rent_VPA_acc           235
vacancia_fisica        126
vacancia_financeira    218
qtd_ativos               0
dtype: int64

In [11]:
len(df)

244

As variáveis `dy_VPA`, `var_VPA`, `rent_VPA_periodo`, `rent_VPA_acc` e `vacancia_financeira` possuem pouquísimos dados, somente 9 (26 para vacancia_financeira) das 244 observações contém algum tipo de informação. 

Essas variáveis serão removidas:

In [12]:
df.drop(['dy_VPA', 'var_VPA', 'rent_VPA_periodo', 'rent_VPA_acc', 'vacancia_financeira'],
        axis = 1, inplace = True)
df.isna().sum()

ticker                   0
nome                     0
tipo_gestao              0
setor                    0
preço                    0
dy                       0
pct_caixa                2
num_cotistas             0
PL                       0
PVPA                     5
liquidez_diaria_avg      6
cagr_dividendo         138
cagr_cota              138
liquidez_diaria          7
div_mes_brl              0
dy_1m                   12
dy_3m_acc               12
dy_6m_acc               12
dy_12m_acc              12
dy_3m_avg               12
dy_6m_avg               12
dy_12m_avg              12
dy_ano                  24
var_preço               12
rent_periodo            12
rent_acc                12
VPA                      0
vacancia_fisica        126
qtd_ativos               0
dtype: int64

As variáveis de CAGR possuem uma grande quantidade de valores nulos, grande parte devido ao fato de o fundo ter menos que 5 anos. Criaremos uma variável categórica `menos_5_anos` para identificar os fundos que possuem ambos os CAGRs ausentes e os valores ausentes serão preenchidos com zero.

`Nota: CAGR vem do inglês "Compound Annual Growth Rate" (Taxa de Crescimento Anual Composta) e é, geralmente, calculada com base em dados dos últimos 5 anos.`

In [13]:
# Insere na posição coluna 4, a fim de manter variáveis categóricas no começo do dataframe
df.insert(4, 'menos_5_anos', False)
df.loc[(df['cagr_dividendo'].isna()) & (df['cagr_cota'].isna()),'menos_5_anos'] = True

In [14]:
df.head()

,ticker,nome,tipo_gestao,setor,menos_5_anos,preço,dy,pct_caixa,num_cotistas,PL,...,dy_3m_avg,dy_6m_avg,dy_12m_avg,dy_ano,var_preço,rent_periodo,rent_acc,VPA,vacancia_fisica,qtd_ativos
0,BRLA11,FUNDO DE INVESTIMENTO IMOBILIÁRIO BRL PROP II,Passiva,Híbrido,True,0.0,0.0000,0.7435,84.0,1.376312e+08,...,0.00,0.00,0.00,NaN,0.00,0.00,0.00,140.98,NaN,0.0
4,SHPH11,Shopping Patio Higienopolis,Passiva,Shoppings,False,807.0,3.4885,0.4134,3105.0,5.013048e+08,...,0.25,0.37,0.28,1.27,0.00,0.24,-0.33,823.23,13.3,1.0
5,ALZR11,Alianza Trust Renda Imobiliária,Ativa,Logística,False,122.0,5.1902,1.8679,93070.0,5.450889e+08,...,0.53,0.49,0.41,2.91,5.65,6.18,-0.60,102.50,0.0,13.0
7,BBRC11,BB Renda Corporativa,Passiva,Outros,False,112.9,10.0089,4.0449,8727.0,1.794707e+08,...,0.77,0.77,0.72,4.61,-3.22,-2.49,-6.74,112.87,5.4,20.0
8,BICR11,INTER TÍTULOS IMOB. FDO INV. IMOB.,Ativa,Títulos e Val. Mob.,True,96.0,3.2083,9.6076,602.0,4.953674e+07,...,0.34,0.30,0.26,1.78,-0.94,-0.62,-4.55,99.07,NaN,0.0


In [15]:
df.loc[:,['cagr_dividendo', 'cagr_cota']] = df[['cagr_dividendo', 'cagr_cota']].fillna(value = 0)

In [16]:
df.isna().sum()

ticker                   0
nome                     0
tipo_gestao              0
setor                    0
menos_5_anos             0
preço                    0
dy                       0
pct_caixa                2
num_cotistas             0
PL                       0
PVPA                     5
liquidez_diaria_avg      6
cagr_dividendo           0
cagr_cota                0
liquidez_diaria          7
div_mes_brl              0
dy_1m                   12
dy_3m_acc               12
dy_6m_acc               12
dy_12m_acc              12
dy_3m_avg               12
dy_6m_avg               12
dy_12m_avg              12
dy_ano                  24
var_preço               12
rent_periodo            12
rent_acc                12
VPA                      0
vacancia_fisica        126
qtd_ativos               0
dtype: int64

Vacância Física é ausente em 126 observações. Alguns são dados ausentes, mas a maioria (75 observações) são fundos de "Títulos e Valores Mobiliários", no qual o conceito de vacância não se aplica.

Esses valores serão preenchidos com zeros:

In [17]:
df[df['vacancia_fisica'].isna()].head()

,ticker,nome,tipo_gestao,setor,menos_5_anos,preço,dy,pct_caixa,num_cotistas,PL,...,dy_3m_avg,dy_6m_avg,dy_12m_avg,dy_ano,var_preço,rent_periodo,rent_acc,VPA,vacancia_fisica,qtd_ativos
0,BRLA11,FUNDO DE INVESTIMENTO IMOBILIÁRIO BRL PROP II,Passiva,Híbrido,True,0.00,0.0000,0.7435,84.0,1.376312e+08,...,0.00,0.00,0.00,NaN,0.00,0.00,0.00,140.98,NaN,0.0
8,BICR11,INTER TÍTULOS IMOB. FDO INV. IMOB.,Ativa,Títulos e Val. Mob.,True,96.00,3.2083,9.6076,602.0,4.953674e+07,...,0.34,0.30,0.26,1.78,-0.94,-0.62,-4.55,99.07,NaN,0.0
11,BTAL11,BTG PACTUAL AGRO LOGÍSTICA,Ativa,Outros,True,98.28,2.2549,33.8504,10544.0,6.132124e+08,...,0.56,0.00,0.00,2.02,-0.28,0.38,-1.13,100.55,NaN,7.0
12,BTCR11,BTG Pactual Crédito Imobiliário,Ativa,Títulos e Val. Mob.,False,93.97,6.8445,5.6180,9427.0,4.681521e+08,...,0.74,0.71,0.60,4.24,0.83,1.61,10.18,97.33,NaN,0.0
19,BZLI11,Brazil Realty,Ativa,Títulos e Val. Mob.,False,14.58,0.0000,2.7259,30.0,4.449849e+08,...,0.00,0.00,0.00,0.00,0.68,0.68,1.14,11.37,NaN,0.0


In [18]:
df.loc[df['vacancia_fisica'].isna(),['ticker', 'setor']].groupby('setor').count()

,ticker
setor,
Hospital,1
Hotel,1
Híbrido,20
Lajes Corporativas,2
Logística,1
Outros,21
Residencial,5
Títulos e Val. Mob.,75


In [19]:
df.loc[:,'vacancia_fisica'] = df['vacancia_fisica'].fillna(value = 0)

In [20]:
df.isna().sum()

ticker                  0
nome                    0
tipo_gestao             0
setor                   0
menos_5_anos            0
preço                   0
dy                      0
pct_caixa               2
num_cotistas            0
PL                      0
PVPA                    5
liquidez_diaria_avg     6
cagr_dividendo          0
cagr_cota               0
liquidez_diaria         7
div_mes_brl             0
dy_1m                  12
dy_3m_acc              12
dy_6m_acc              12
dy_12m_acc             12
dy_3m_avg              12
dy_6m_avg              12
dy_12m_avg             12
dy_ano                 24
var_preço              12
rent_periodo           12
rent_acc               12
VPA                     0
vacancia_fisica         0
qtd_ativos              0
dtype: int64

Existem 5 observações de PVPA (Preço dividido por Valor Patrimonial) ausentes. Em geral são fundos pequenos, com baixa liquidez que serão removidos da análise.

In [21]:
df.loc[df['PVPA'].isna(), ['ticker', 'preço', 'PL', 'liquidez_diaria_avg', 'liquidez_diaria']]

,ticker,preço,PL,liquidez_diaria_avg,liquidez_diaria
0,BRLA11,0.00,1.376312e+08,NaN,NaN
84,PABY11,12.58,-6.281734e+06,3090.38,54.0
112,SPVJ11,0.00,5.302450e+08,NaN,NaN
140,BBIM11,0.00,6.462463e+07,NaN,NaN
274,RBCB11,19.00,0.000000e+00,NaN,161.0


In [22]:
df.dropna(subset = ['PVPA'], axis = 0, inplace = True)

In [23]:
df.isna().sum()

ticker                  0
nome                    0
tipo_gestao             0
setor                   0
menos_5_anos            0
preço                   0
dy                      0
pct_caixa               0
num_cotistas            0
PL                      0
PVPA                    0
liquidez_diaria_avg     2
cagr_dividendo          0
cagr_cota               0
liquidez_diaria         4
div_mes_brl             0
dy_1m                  12
dy_3m_acc              12
dy_6m_acc              12
dy_12m_acc             12
dy_3m_avg              12
dy_6m_avg              12
dy_12m_avg             12
dy_ano                 21
var_preço              12
rent_periodo           12
rent_acc               12
VPA                     0
vacancia_fisica         0
qtd_ativos              0
dtype: int64

As demais variáveis sem dados terão seus valores definidos como zeros.

In [25]:
df.fillna(value = 0, inplace = True)
df.isna().sum()

ticker                 0
nome                   0
tipo_gestao            0
setor                  0
menos_5_anos           0
preço                  0
dy                     0
pct_caixa              0
num_cotistas           0
PL                     0
PVPA                   0
liquidez_diaria_avg    0
cagr_dividendo         0
cagr_cota              0
liquidez_diaria        0
div_mes_brl            0
dy_1m                  0
dy_3m_acc              0
dy_6m_acc              0
dy_12m_acc             0
dy_3m_avg              0
dy_6m_avg              0
dy_12m_avg             0
dy_ano                 0
var_preço              0
rent_periodo           0
rent_acc               0
VPA                    0
vacancia_fisica        0
qtd_ativos             0
dtype: int64

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 239 entries, 4 to 353
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ticker               239 non-null    object 
 1   nome                 239 non-null    object 
 2   tipo_gestao          239 non-null    object 
 3   setor                239 non-null    object 
 4   menos_5_anos         239 non-null    bool   
 5   preço                239 non-null    float64
 6   dy                   239 non-null    float64
 7   pct_caixa            239 non-null    float64
 8   num_cotistas         239 non-null    float64
 9   PL                   239 non-null    float64
 10  PVPA                 239 non-null    float64
 11  liquidez_diaria_avg  239 non-null    float64
 12  cagr_dividendo       239 non-null    float64
 13  cagr_cota            239 non-null    float64
 14  liquidez_diaria      239 non-null    float64
 15  div_mes_brl          239 non-null    flo

Dessa forma, é finalizado o tratamento de dados faltantes.

O Dataframe resultante possui **239 observações com 29 variáveis**.

Uma cópia desse dataframe será salva na pasta de dados para poder ser utilizada em outras análises.

In [27]:
df.to_csv("data/dados_FIIs_processados.csv")